### Plant Classifier Algorithm ###
**Declaration for all libraries**
*Important to run R7_External_Rohit_Plant_Classifier_Data_Load. *
*Above script loads data from competition to the COLAB machine*


In [1]:
import tensorflow as tf
import numpy as np # MATRIX OPERATIONS
import pandas as pd # EFFICIENT DATA STRUCTURES
import matplotlib.pyplot as plt # GRAPHING AND VISUALIZATIONS
import math # MATHEMATICAL OPERATIONS
import cv2 # IMAGE PROCESSING - OPENCV
from glob import glob # FILE OPERATIONS
import itertools

# KERAS AND SKLEARN MODULES
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import BatchNormalization
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,CSVLogger

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

Using TensorFlow backend.


In [0]:
scale = 256
seed = 7

### 1. Read the images and generate the train and test dataset (5 points)
### 2. Divide the data set into Train and validation data sets


* Step 1 is to read the data from the Folders
* We will use Image Augmentation
* Step 2 We will use sub classes of Image augmentation to split data into Training and Validation Dataset

In [0]:
from keras.preprocessing.image import ImageDataGenerator

In [0]:
data = ImageDataGenerator(validation_split=0.2, 
                          rescale=1/255,
                          samplewise_center=False, # set input mean to 0 over the sample
                          samplewise_std_normalization=False,  # divide inputs by std of the sample
                          rotation_range=90,       # randomly rotate images in the range (degrees, 0 to 180)
                          width_shift_range=0.2,   # randomly shift images horizontally (fraction of total width)
                          height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
                          fill_mode='reflect',     # filling the area outside
                          zoom_range=0.4,          # random zoom
                          horizontal_flip=True,    # randomly flip images
                          vertical_flip=True)      # randomly flip images

In [5]:
train_generator = data.flow_from_directory(directory="train/", 
                                           target_size=(scale,scale),
                                           class_mode="categorical", 
                                           shuffle=True, 
                                           batch_size=8, 
                                           subset='training')

Found 3803 images belonging to 12 classes.


In [6]:
val_generator = data.flow_from_directory(directory="train/", 
                                           target_size=(scale,scale),
                                           class_mode="categorical", 
                                           shuffle=True, 
                                           batch_size=8, 
                                           subset='validation')

Found 947 images belonging to 12 classes.


### 3. Initialize & build the model (10 points)

* Step 1 We will use Transfer learning to use pre built models

In [0]:
from keras.applications import ResNet50
from keras.models import Model

In [8]:
conv_base = ResNet50(include_top=False, weights='imagenet', input_shape=(256,256,3), pooling="avg")

Instructions for updating:
Colocations handled automatically by placer.
94658560/94653016 [==============================] - 1s 0us/step


In [0]:
for layer in conv_base.layers:
    layer.trainable = False

* Step 2 Optimize the Model

In [10]:
x = conv_base.output
x = Dense(512, activation='relu')(x) # let's add a fully-connected layer
x = Dropout(0.25)(x)
x = Dense(128, activation='relu')(x) # let's add a fully-connected layer
x = Dropout(0.25)(x)
predictions = Dense(12, activation='softmax')(x)

# this is the model we will train
m = Model(inputs=conv_base.input, outputs=predictions)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
m.compile(loss='binary_crossentropy', optimizer="sgd", metrics=['accuracy'])

### 5. Predict the accuracy for both train and validation data (7 points)

In [12]:
m_output =  m.fit_generator(train_generator, epochs=8, steps_per_epoch=1000 // 50,
                            validation_data=val_generator, validation_steps=600 // 50,
                            verbose = 1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/8
20/20 [==============================] - 102s 5s/step - loss: 0.3036 - acc: 0.9161 - val_loss: 0.3135 - val_acc: 0.9167
Epoch 2/8
20/20 [==============================] - 96s 5s/step - loss: 0.3115 - acc: 0.9167 - val_loss: 0.2947 - val_acc: 0.9167
Epoch 3/8
20/20 [==============================] - 96s 5s/step - loss: 0.3021 - acc: 0.9167 - val_loss: 0.3080 - val_acc: 0.9167
Epoch 4/8
20/20 [==============================] - 96s 5s/step - loss: 0.2909 - acc: 0.9167 - val_loss: 0.2964 - val_acc: 0.9167
Epoch 5/8
20/20 [==============================] - 96s 5s/step - loss: 0.2954 - acc: 0.9167 - val_loss: 0.2974 - val_acc: 0.9167
Epoch 6/8
20/20 [==============================] - 96s 5s/step - loss: 0.2970 - acc: 0.9167 - val_loss: 0.2965 - val_acc: 0.9167
Epoch 7/8
20/20 [==============================] - 96s 5s/step - loss: 0.2909 - acc: 0.9167 - val_loss: 0.3057 - val_acc: 0.9167
Epoch 8/8
20/20 [==============================]

*Step3 to Check the predictions of test folder*

In [0]:
path_to_test = '/test'
pics = glob(path_to_test)

testimages = []
tests = []
count=1
num = len(pics)

for i in pics:
    print(str(count)+'/'+str(num),end='\r')
    tests.append(i.split('/')[-1])
    testimages.append(cv2.resize(cv2.imread(i),(scale,scale)))
    count = count + 1

testimages = np.asarray(testimages)